## This was run in AWS

In [28]:
# !pip install seaborn
# ! pip install kmeans-pytorch

In [29]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch

# from sklearn.cluster import KMeans
from kmeans_pytorch import kmeans
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score, fowlkes_mallows_score # TODO read about these scoring mechanisms
from sklearn.decomposition import PCA

In [30]:
df = pd.read_pickle("./data/clean_2020articles_w_title_embeddings.pkl")

In [37]:
embedding = df['bert_sentence_embedding_title_only'].squeeze()

In [19]:
def eval_cluster(embedding, kmeans):
    y_pred = kmeans.fit_predict(embedding)

    # Evaluate the performance using ARI, NMI, and FMI
    ari = adjusted_rand_score(df["target"], y_pred)
    nmi = normalized_mutual_info_score(df["target"], y_pred)
    fmi = fowlkes_mallows_score(df["target"], y_pred)

    # Print Metrics scores
    print("Adjusted Rand Index (ARI): {:.3f}".format(ari))
    print("Normalized Mutual Information (NMI): {:.3f}".format(nmi))
    print("Fowlkes-Mallows Index (FMI): {:.3f}".format(fmi))

In [20]:
def dimension_reduction(embedding, method):

    pca = PCA(n_components=2, random_state=42)

    pca_vecs = pca.fit_transform(embedding)

    # save our two dimensions into x0 and x1
    x0 = pca_vecs[:, 0]
    x1 = pca_vecs[:, 1]

    df[f'x0_{method}'] = x0
    df[f'x1_{method}'] = x1

In [21]:
def plot_pca(x0_name, x1_name, cluster_name, method):

    plt.figure(figsize=(12, 7))

    plt.title(f"TF-IDF + KMeans 20newsgroup clustering with {method}", fontdict={"fontsize": 18})
    plt.xlabel("X0", fontdict={"fontsize": 16})
    plt.ylabel("X1", fontdict={"fontsize": 16})

    sns.scatterplot(data=df, x=x0_name, y=x1_name, hue=cluster_name, palette="viridis")
    plt.show()

In [32]:
# method = 'bert_sentence_title_embedding'
# # initialize kmeans with 3 centroids
# kmeans = KMeans(n_clusters=3, random_state=42)


# # fit the model
# kmeans.fit(embedding)

# # store cluster labels in a variable
# clusters = kmeans.labels_

# # Assign clusters to our dataframe
# clusters_result_name = f'cluster_bert_embeddings'
# df['bert_clusters'] = clusters

# eval_cluster(embedding)

# dimension_reduction(embedding, method)

# plot_pca(f'x0_{method}', f'x1_{method}', cluster_name=clusters_result_name, method=method)

In [41]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

cluster_ids_x, cluster_centers = kmeans(
    X=embedding.values, num_clusters=3, distance='euclidean', device=device
)

running k-means on cuda:0..


AttributeError: 'numpy.ndarray' object has no attribute 'float'